# Data Lake on S3

In [10]:
from pyspark.sql import SparkSession


import os
import configparser

In [11]:
import databricks.koalas as ks

# Make sure that your AWS credentials are loaded as env vars

In [12]:
config = configparser.ConfigParser()

#Normally this file should be in ~/.aws/credentials
config.read('dl.cfg')

os.environ['AWS_ACCESS_KEY_ID']=config.get('AWS','AWS_ACCESS_KEY_ID')
os.environ['AWS_SECRET_ACCESS_KEY']=config.get('AWS','AWS_SECRET_ACCESS_KEY')

# Create spark session with hadoop-aws package

In [13]:
spark = SparkSession.builder\
                    .config("spark.jars.packages","org.apache.hadoop:hadoop-aws:2.7.0")\
                     .getOrCreate()

# Load data from S3

In [14]:
input_data = "s3a://udacity-dend/"
output_data = "s3a://project4dend/"
    

In [15]:
# data/song_data/A/B/C/*.json           THIS WORKS 

kdf = ks.read_json("s3a://udacity-dend/song_data/*/*/*/*.json")   # BUT THIS ONE DOESN'T 

Py4JJavaError: An error occurred while calling o275.load.
: com.amazonaws.services.s3.model.AmazonS3Exception: Status Code: 403, AWS Service: Amazon S3, AWS Request ID: BC93B15580C59563, AWS Error Code: null, AWS Error Message: Forbidden, S3 Extended Request ID: evQSJc23XcR54NuSpmR6X0oG0/GNGRSsZ1aLtkin8Se/sgeEG3BiRWoXvLwZDiyu7J95ll4P+ok=
	at com.amazonaws.http.AmazonHttpClient.handleErrorResponse(AmazonHttpClient.java:798)
	at com.amazonaws.http.AmazonHttpClient.executeHelper(AmazonHttpClient.java:421)
	at com.amazonaws.http.AmazonHttpClient.execute(AmazonHttpClient.java:232)
	at com.amazonaws.services.s3.AmazonS3Client.invoke(AmazonS3Client.java:3528)
	at com.amazonaws.services.s3.AmazonS3Client.getObjectMetadata(AmazonS3Client.java:976)
	at com.amazonaws.services.s3.AmazonS3Client.getObjectMetadata(AmazonS3Client.java:956)
	at org.apache.hadoop.fs.s3a.S3AFileSystem.getFileStatus(S3AFileSystem.java:892)
	at org.apache.hadoop.fs.s3a.S3AFileSystem.listStatus(S3AFileSystem.java:734)
	at 

In [7]:
kdf.dtypes

artist_id            object
artist_latitude     float64
artist_location      object
artist_longitude    float64
artist_name          object
duration            float64
num_songs             int64
song_id              object
title                object
year                  int64
dtype: object

In [8]:
kdf.to_spark().printSchema()
kdf.head()

root
 |-- artist_id: string (nullable = true)
 |-- artist_latitude: double (nullable = true)
 |-- artist_location: string (nullable = true)
 |-- artist_longitude: double (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- duration: double (nullable = true)
 |-- num_songs: long (nullable = true)
 |-- song_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- year: long (nullable = true)



,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARNF6401187FB57032,40.79086,"New York, NY [Manhattan]",-73.96644,Sophie B. Hawkins,305.16200,1,SONWXQJ12A8C134D94,The Ballad Of Sleeping Beauty,1994
1,ARLTWXK1187FB5A3F8,32.74863,"Fort Worth, TX",-97.32925,King Curtis,326.00771,1,SODREIN12A58A7F2E5,A Whiter Shade Of Pale (Live @ Fillmore West),0
2,ARPFHN61187FB575F6,41.88415,"Chicago, IL",-87.63241,Lupe Fiasco,279.97995,1,SOWQTQZ12A58A7B63E,Streets On Fire (Explicit Album Version),0
3,AR0IAWL1187B9A96D0,8.41770,Panama,-80.11278,Danilo Perez,197.19791,1,SONSKXP12A8C13A2C9,Native Soul,2003
4,AREVWGE1187B9B890A,-13.44200,Noci (BA),-41.99520,Bitter End,282.43546,1,SOFCHDR12AB01866EF,Living Hell,0


# saving the data

In [9]:


songs_table = (ks.sql('''
               SELECT 
               DISTINCT
               row_number() over (ORDER BY year,title,artist_id) id,
               title,
               artist_id,
               year,
               duration
               FROM 
                   {kdf}''')
              )

##songs_table.to_spark().withColumn("id", monotonicallyIncreasingId())
songs_table

,id,title,artist_id,year,duration
0,1,A Whiter Shade Of Pale (Live @ Fillmore West),ARLTWXK1187FB5A3F8,0,326.00771
1,2,Der Kleine Dompfaff,ARJIE2Y1187B994AB7,0,152.92036
2,3,Living Hell,AREVWGE1187B9B890A,0,282.43546
3,4,Midnight Star,ARULZCI1241B9C8611,0,335.51628
4,5,Music is what we love,AR051KA1187B98B2FF,0,261.51138
5,6,Streets On Fire (Explicit Album Version),ARPFHN61187FB575F6,0,279.97995
6,7,The Ballad Of Sleeping Beauty,ARNF6401187FB57032,1994,305.16200
7,8,Prognosis,ARWB3G61187FB49404,2000,363.85914
8,9,Intro,AR558FS1187FB45658,2003,75.67628
9,10,Native Soul,AR0IAWL1187B9A96D0,2003,197.19791


In [10]:
songs_table.shape

(12, 5)

In [11]:
songs_table.head()

,id,title,artist_id,year,duration
0,1,A Whiter Shade Of Pale (Live @ Fillmore West),ARLTWXK1187FB5A3F8,0,326.00771
1,2,Der Kleine Dompfaff,ARJIE2Y1187B994AB7,0,152.92036
2,3,Living Hell,AREVWGE1187B9B890A,0,282.43546
3,4,Midnight Star,ARULZCI1241B9C8611,0,335.51628
4,5,Music is what we love,AR051KA1187B98B2FF,0,261.51138


### Apache Parquet Introduction

Apache Parquet is a columnar file format that provides optimizations to speed up queries and is a far more efficient file format than CSV or JSON, supported by many data processing systems.




### Spark Write DataFrame to Parquet file format

Using spark.write.parquet() function we can write Spark DataFrame to Parquet file.




### Spark parquet partition – Improving performance

> Partitioning is a feature of many databases and data processing frameworks and it is key to make jobs work at scale. We can do a parquet file partition using spark partitionBy function.

> Parquet Partition creates a folder hierarchy for each spark partition; we have mentioned the first partition as year followed by artist_id hence, it creates a artist_id folder inside the year folder.

In [13]:
 #new write  . 
songs_table.to_spark().write.mode('overwrite').partitionBy("year", "artist_id").parquet('songs/')


In [ ]:
## ---------------- 

In [16]:
kdf_logs = ks.read_json("data/*.json")

In [17]:
kdf_logs.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Harmonia,Logged In,Ryan,M,0,Smith,655.77751,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541017e+12,583,Sehr kosmisch,200,1542241826796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26
1,The Prodigy,Logged In,Ryan,M,1,Smith,260.07465,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541017e+12,583,The Big Gundown,200,1542242481796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26
2,Train,Logged In,Ryan,M,2,Smith,205.45261,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541017e+12,583,Marry Me,200,1542242741796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26
3,None,Logged In,Wyatt,M,0,Scott,NaN,free,"Eureka-Arcata-Fortuna, CA",GET,Home,1.540872e+12,563,None,200,1542247071796,Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7....,9
4,None,Logged In,Austin,M,0,Rosales,NaN,free,"New York-Newark-Jersey City, NY-NJ-PA",GET,Home,1.541060e+12,521,None,200,1542252577796,Mozilla/5.0 (Windows NT 6.1; rv:31.0) Gecko/20...,12


In [ ]:
users_table = spark

In [18]:
users_table =(ks.sql('''
               SELECT 
               DISTINCT
               userId as user_id,
               firstName,
               lastName,
               gender,
               level
               FROM 
                   {kdf_logs}
                    ''')
              )

In [19]:
users_table.head()

,user_id,firstName,lastName,gender,level
0,98,Jordyn,Powell,F,free
1,34,Evelin,Ayala,F,free
2,85,Kinsley,Young,F,paid
3,38,Gianna,Jones,F,free
4,85,Kinsley,Young,F,free


In [20]:
(users_table
 .to_spark()
 .write
 .mode('overwrite')
 .parquet('users/')
)

In [21]:
artists_table =(ks.sql('''
               SELECT 
               DISTINCT
               artist_id,artist_latitude,artist_location,artist_longitude,artist_name
               FROM 
                   {kdf}''')
              )
artists_table.head()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name
0,ARULZCI1241B9C8611,NaN,,NaN,Luna Orbit Project
1,AR0IAWL1187B9A96D0,8.41770,Panama,-80.11278,Danilo Perez
2,ARWB3G61187FB49404,NaN,"Hamilton, Ohio",NaN,Steve Morse
3,ARLTWXK1187FB5A3F8,32.74863,"Fort Worth, TX",-97.32925,King Curtis
4,AREVWGE1187B9B890A,-13.44200,Noci (BA),-41.99520,Bitter End


In [22]:
artists_table.head()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name
0,ARULZCI1241B9C8611,NaN,,NaN,Luna Orbit Project
1,AR0IAWL1187B9A96D0,8.41770,Panama,-80.11278,Danilo Perez
2,ARWB3G61187FB49404,NaN,"Hamilton, Ohio",NaN,Steve Morse
3,ARLTWXK1187FB5A3F8,32.74863,"Fort Worth, TX",-97.32925,King Curtis
4,AREVWGE1187B9B890A,-13.44200,Noci (BA),-41.99520,Bitter End


In [23]:
(artists_table
 .to_spark()
 .write
 .mode('overwrite')
 .partitionBy("artist_name")
 .parquet('artists/')
)

In [24]:
from pyspark.sql.functions import udf, col
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, date_format
from datetime import datetime

logs = spark.read.json("data/*.json")
logs =  logs.filter(logs.page == 'NextSong')
##so we can use sql lines.
logs.createOrReplaceTempView("logs_view")



In [26]:



time_table = spark.sql("""
                            SELECT 
                            X.start_time_sub as start_time,
                            hour(X.start_time_sub) as hour,
                            dayofmonth(X.start_time_sub) as day,
                            weekofyear(X.start_time_sub) as week,
                            month(X.start_time_sub) as month,
                            year(X.start_time_sub) as year,
                            dayofweek(X.start_time_sub) as weekday
                            FROM
                            (SELECT to_timestamp(L.ts/1000) as start_time_sub
                            FROM logs_view L
                            WHERE L.ts IS NOT NULL
                            ) X
                        """)

In [27]:
(time_table
 .write
 .mode('overwrite')
 .partitionBy("year", "month")
 .parquet('time/')
)

In [28]:
kdf.to_spark().createOrReplaceTempView("songs_view")

In [30]:
songplays_table = spark.sql("""
                                SELECT monotonically_increasing_id() as songplay_id,
                                to_timestamp(L.ts/1000) as start_time,
                                month(to_timestamp(L.ts/1000)) as month,
                                year(to_timestamp(L.ts/1000)) as year,
                                L.userId as user_id,
                                L.level as level,
                                S.song_id as song_id,
                                S.artist_id as artist_id,
                                L.sessionId as session_id,
                                L.location as location,
                                L.userAgent as user_agent
                                FROM logs_view L
                                JOIN songs_view S on L.artist = S.artist_name and L.song = S.title
                            """)

In [31]:
(songplays_table
 .write
 .mode('overwrite')
 .partitionBy("year", "month")
 .parquet('songplays/')
)

## Deployment

1. Install `awscli`

2. run `aws configure` 
    * AWS Access Key ID : 
    * AWS Secret Access Key : 
    * Default region name: `us-west-2`
    * Default output format : `json`
    
3. **copy all the necessary files to an s3 bucket**

    * Ex: `aws s3 cp <filename> s3://<bucket_name>`


4. **Run EMR create script with the etl job**

```
aws emr create-cluster --name "Spark cluster with step" \
    --release-label emr-5.30.1 \
    --applications Name=Spark \
    --log-uri s3://dendsparktutorial/logs/ \
    --ec2-attributes KeyName=emr-key \
    --instance-type m5.xlarge \
    --instance-count 3 \
    --bootstrap-actions Path=s3://dendsparktutorial/emr_bootstrap.sh \
    --steps Type=Spark,Name="Spark program",ActionOnFailure=CONTINUE,Args=[--deploy-mode,cluster,--master,yarn,s3://dendsparktutorial/src/koalas_etl.py] \
    --use-default-roles \
    --auto-terminate
```